In [ ]:
import pandas as pd
import pyomo.environ as pyo
from pyomo.opt import SolverStatus, TerminationCondition
import numpy as np
import time

print("=== MODELO OPTIMIZADO - 5 POLÍGONOS MÁS CERCANOS AL ALMACÉN ===")

# === DATOS ACTUALIZADOS DESDE ARCHIVOS EXCEL ===
nombres_especies = [
    'lechuguilla', 'salmiana', 'scabra', 'striata',
    'cantabrigiensis', 'engelmani', 'robusta', 'streptacanta', 
    'laevigata', 'yucca'
]

# 🎯 LOS 5 POLÍGONOS MÁS CERCANOS AL ALMACÉN (POLÍGONO 18)
poligonos_filtrados = ['poligono13', 'poligono14', 'poligono17', 'poligono23', 'poligono24']

# ⏱️ TIEMPOS REALES DESDE ALMACÉN (POLÍGONO 18) A DESTINOS
tiempo_almacen_poligono = {
    'poligono24': 0.0067,    # MÁS CERCANO - 0.4 minutos
    'poligono17': 0.0075,    # 0.45 minutos  
    'poligono13': 0.0123,    # 0.74 minutos
    'poligono23': 0.0130,    # 0.78 minutos
    'poligono14': 0.0138,    # 0.83 minutos
}

# 📊 DEMANDA REAL POR (ESPECIE, POLÍGONO) DESDE ARCHIVO EXCEL
demanda_filtrada = {
    # POLÍGONO 13 (2,829 plantas)
    ('lechuguilla', 'poligono13'): 178,
    ('salmiana', 'poligono13'): 848,
    ('scabra', 'poligono13'): 178,
    ('striata', 'poligono13'): 178,
    ('cantabrigiensis', 'poligono13'): 211,
    ('engelmani', 'poligono13'): 162,
    ('robusta', 'poligono13'): 313,
    ('streptacanta', 'poligono13'): 275,
    ('laevigata', 'poligono13'): 373,
    ('yucca', 'poligono13'): 113,
    
    # POLÍGONO 14 (3,133 plantas)
    ('lechuguilla', 'poligono14'): 197,
    ('salmiana', 'poligono14'): 939,
    ('scabra', 'poligono14'): 197,
    ('striata', 'poligono14'): 197,
    ('cantabrigiensis', 'poligono14'): 233,
    ('engelmani', 'poligono14'): 179,
    ('robusta', 'poligono14'): 347,
    ('streptacanta', 'poligono14'): 305,
    ('laevigata', 'poligono14'): 413,
    ('yucca', 'poligono14'): 126,
    
    # POLÍGONO 17 (3,202 plantas)
    ('lechuguilla', 'poligono17'): 202,
    ('salmiana', 'poligono17'): 959,
    ('scabra', 'poligono17'): 202,
    ('striata', 'poligono17'): 202,
    ('cantabrigiensis', 'poligono17'): 238,
    ('engelmani', 'poligono17'): 183,
    ('robusta', 'poligono17'): 354,
    ('streptacanta', 'poligono17'): 312,
    ('laevigata', 'poligono17'): 422,
    ('yucca', 'poligono17'): 128,
    
    # POLÍGONO 23 (2,897 plantas)
    ('lechuguilla', 'poligono23'): 182,
    ('salmiana', 'poligono23'): 868,
    ('scabra', 'poligono23'): 182,
    ('striata', 'poligono23'): 182,
    ('cantabrigiensis', 'poligono23'): 216,
    ('engelmani', 'poligono23'): 166,
    ('robusta', 'poligono23'): 321,
    ('streptacanta', 'poligono23'): 282,
    ('laevigata', 'poligono23'): 382,
    ('yucca', 'poligono23'): 116,
    
    # POLÍGONO 24 (2,954 plantas) - MÁS CERCANO
    ('lechuguilla', 'poligono24'): 186,
    ('salmiana', 'poligono24'): 885,
    ('scabra', 'poligono24'): 186,
    ('striata', 'poligono24'): 186,
    ('cantabrigiensis', 'poligono24'): 220,
    ('engelmani', 'poligono24'): 169,
    ('robusta', 'poligono24'): 327,
    ('streptacanta', 'poligono24'): 288,
    ('laevigata', 'poligono24'): 389,
    ('yucca', 'poligono24'): 118,
}

demanda_total = sum(demanda_filtrada.values())
print(f"✅ Demanda total: {demanda_total:,} plantas")
print(f"📍 Almacén: POLÍGONO 18")
print(f"🎯 Destinos: {', '.join(poligonos_filtrados)}")
print(f"⚡ Polígono más cercano: 24 ({tiempo_almacen_poligono['poligono24']:.4f} horas)")
print(f"🐌 Polígono más lejano: 14 ({tiempo_almacen_poligono['poligono14']:.4f} horas)")

# === FUNCIÓN PARA CREAR MODELO CON PARÁMETROS VARIABLES ===
def crear_modelo_optimizacion(
    # Costos
    costo_plantacion=20,        
    costo_logistico=3.0,        
    costo_viaje_fijo=4500,      
    costo_compra=26,            
    penalizacion_tiempo=500,     
    penalizacion_deterioro=25,  
    
    # Capacidades operativas
    capacidad_almacen=8000,    
    plantas_por_viaje=524,      
    max_viajes_dia=5,           
    horas_laborales=6,          
    tiempo_carga=0.5,           
    limite_compra_dia=8000,     
    
    # Parámetros de tiempo
    tiempo_min_maduracion=3,    
    tiempo_max_almacen=7,       
    
    # Límites de proveedores
    limite_moctezuma=3500,
    limite_venado=3000,
    limite_laguna_seca=3000,
    
    # Control de impresión
    verbose=True
):
    """
    Modelo optimizado para los 5 polígonos más cercanos al almacén (polígono 18)
    """
    if verbose:
        print(f"\n🔧 CREANDO MODELO OPTIMIZADO:")
        print(f"   📍 Almacén: POLÍGONO 18")
        print(f"   🎯 Destinos: 5 más cercanos ({', '.join([p.split('o')[1] for p in poligonos_filtrados])})")
        print(f"   💰 Costos: Plantación=${costo_plantacion}, Viaje=${costo_viaje_fijo}, Compra=${costo_compra}")
        print(f"   📦 Capacidades: Almacén={capacidad_almacen:,}, Viajes/día={max_viajes_dia}")
        print(f"   ⏰ Tiempo: {horas_laborales}h laborales, maduración {tiempo_min_maduracion}-{tiempo_max_almacen} días")
        print(f"   🌱 Demanda total: {demanda_total:,} plantas")
    
    # Límites de proveedores
    limite_pedido_proveedor = {
        'moctezuma': limite_moctezuma,
        'venado': limite_venado,
        'laguna_seca': limite_laguna_seca,
    }
    
    # === CREAR MODELO ===
    model = pyo.ConcreteModel()
    
    # Conjuntos
    model.S = pyo.Set(initialize=nombres_especies)
    model.J = pyo.Set(initialize=['moctezuma', 'venado', 'laguna_seca'])
    model.P = pyo.Set(initialize=poligonos_filtrados)
    model.T = pyo.Set(initialize=range(1, 29))  # 28 días
    
    # Parámetros
    model.C = pyo.Param(model.S, model.J, initialize=lambda m, s, j: costo_compra)
    model.demanda = pyo.Param(model.S, model.P, initialize=demanda_filtrada, default=0)
    model.tiempo_almacen = pyo.Param(model.P, initialize=tiempo_almacen_poligono, default=999)
    
    # === VARIABLES ===
    model.X = pyo.Var(model.S, model.J, model.T, domain=pyo.NonNegativeIntegers)  # Compras
    model.E = pyo.Var(model.S, model.P, model.T, domain=pyo.NonNegativeIntegers)  # Envíos desde almacén
    model.A = pyo.Var(model.T, model.S, domain=pyo.NonNegativeIntegers)           # Arribos al almacén
    model.I = pyo.Var(model.S, model.T, domain=pyo.NonNegativeIntegers)           # Inventario en almacén
    model.V = pyo.Var(model.P, model.T, domain=pyo.NonNegativeIntegers)           # Viajes desde polígono 18
    model.Z = pyo.Var(model.T, domain=pyo.Binary)                                 # Día con compras
    
    # Variables de tiempo
    model.TiempoTotalDia = pyo.Var(model.T, domain=pyo.NonNegativeReals)
    model.TiempoExtra = pyo.Var(model.T, domain=pyo.NonNegativeReals)
    
    # Variables de control de calidad
    model.PlantasDeterioro = pyo.Var(model.S, model.T, domain=pyo.NonNegativeIntegers)
    
    # === FUNCIÓN OBJETIVO ===
    def objective_rule(m):
        # Costos principales
        costo_compra_total = sum(m.C[s, j] * m.X[s, j, t] for s in m.S for j in m.J for t in m.T)
        costo_plantacion_total = costo_plantacion * sum(m.E[s, p, t] for s in m.S for p in m.P for t in m.T)
        costo_viajes_total = costo_viaje_fijo * sum(m.V[p, t] for p in m.P for t in m.T)
        
        # Costos de tiempo de viaje (desde polígono 18)
        costo_tiempo_viaje = costo_logistico * sum(
            m.tiempo_almacen[p] * 2 * m.V[p, t]  # Ida y vuelta
            for p in m.P for t in m.T
        )
        
        # Penalizaciones
        costo_tiempo_extra = penalizacion_tiempo * sum(m.TiempoExtra[t] for t in m.T)
        costo_deterioro_total = penalizacion_deterioro * sum(m.PlantasDeterioro[s, t] for s in m.S for t in m.T)
        
        return (costo_compra_total + costo_plantacion_total + costo_viajes_total + 
                costo_tiempo_viaje + costo_tiempo_extra + costo_deterioro_total)
    
    model.OBJ = pyo.Objective(rule=objective_rule, sense=pyo.minimize)
    
    # === RESTRICCIONES ===
    
    # 1. Balance de inventario en almacén (polígono 18)
    def balance_inventario_rule(m, s, t):
        if t == 1:
            return m.I[s, t] == m.A[t, s] - sum(m.E[s, p, t] for p in m.P)
        else:
            return m.I[s, t] == m.I[s, t - 1] + m.A[t, s] - sum(m.E[s, p, t] for p in m.P)
    model.BalanceInventario = pyo.Constraint(model.S, model.T, rule=balance_inventario_rule)
    
    # 2. Capacidad del almacén (polígono 18)
    def capacidad_inventario_rule(m, t):
        return sum(m.I[s, t] for s in m.S) <= capacidad_almacen
    model.CapacidadInventario = pyo.Constraint(model.T, rule=capacidad_inventario_rule)
    
    # 3. Capacidad de transporte por viaje
    def capacidad_transporte_viaje_rule(m, p, t):
        return sum(m.E[s, p, t] for s in m.S) <= plantas_por_viaje * m.V[p, t]
    model.CapacidadTransporteViaje = pyo.Constraint(model.P, model.T, rule=capacidad_transporte_viaje_rule)
    
    # 4. Requerir al menos un viaje si hay envíos
    def minimo_viajes_rule(m, p, t):
        total_envios = sum(m.E[s, p, t] for s in m.S)
        return total_envios <= plantas_por_viaje * m.V[p, t]
    model.MinimoViajes = pyo.Constraint(model.P, model.T, rule=minimo_viajes_rule)
    
    # 5. Límite de viajes por día
    def limite_viajes_por_dia_rule(m, t):
        return sum(m.V[p, t] for p in m.P) <= max_viajes_dia
    model.LimiteViajesDia = pyo.Constraint(model.T, rule=limite_viajes_por_dia_rule)
    
    # 6. Límite de compras por proveedor
    def limite_compra_proveedor_rule(m, j, t):
        limite = limite_pedido_proveedor.get(j, limite_compra_dia)
        return sum(m.X[s, j, t] for s in m.S) <= limite * m.Z[t]
    model.LimiteCompraProveedor = pyo.Constraint(model.J, model.T, rule=limite_compra_proveedor_rule)
    
    # 7. Límite total de compras diarias
    def limite_compras_diarias_rule(m, t):
        return sum(m.X[s, j, t] for s in m.S for j in m.J) <= limite_compra_dia * m.Z[t]
    model.LimiteCompraDiaria = pyo.Constraint(model.T, rule=limite_compras_diarias_rule)
    
    # 8. Entrada al almacén (tiempo de maduración)
    def entrada_almacen_rule(m, t, s):
        llegadas = 0
        for tau in range(1, t):
            if tiempo_min_maduracion <= t - tau <= tiempo_max_almacen:
                llegadas += sum(m.X[s, j, tau] for j in m.J)
        return m.A[t, s] == llegadas
    model.EntradaAlmacen = pyo.Constraint(model.T, model.S, rule=entrada_almacen_rule)
    
    # 9. Satisfacer demanda total
    def satisfacer_demanda_rule(m, s, p):
        demanda_requerida = m.demanda[s, p]
        return sum(m.E[s, p, t] for t in m.T) >= demanda_requerida
    model.SatisfacerDemanda = pyo.Constraint(model.S, model.P, rule=satisfacer_demanda_rule)
    
    # 10. Cálculo del tiempo total usado por día
    def tiempo_total_dia_rule(m, t):
        tiempo_viajes = sum(m.tiempo_almacen[p] * 2 * m.V[p, t] for p in m.P)
        tiempo_carga_total = tiempo_carga * sum(m.V[p, t] for p in m.P)
        return m.TiempoTotalDia[t] == tiempo_viajes + tiempo_carga_total
    model.TiempoTotalDiaConstr = pyo.Constraint(model.T, rule=tiempo_total_dia_rule)
    
    # 11. Cálculo de tiempo extra
    def tiempo_extra_rule(m, t):
        return m.TiempoExtra[t] >= m.TiempoTotalDia[t] - horas_laborales
    model.TiempoExtraCalculo = pyo.Constraint(model.T, rule=tiempo_extra_rule)
    
    # 12. Control básico de deterioro
    def control_deterioro_rule(m, s, t):
        if t > tiempo_max_almacen + 3:
            return m.PlantasDeterioro[s, t] >= 0.05 * m.I[s, t]
        else:
            return m.PlantasDeterioro[s, t] == 0
    model.ControlDeterioro = pyo.Constraint(model.S, model.T, rule=control_deterioro_rule)
    
    return model

# === FUNCIÓN PARA RESOLVER CON LÍMITE DE TIEMPO ===
def resolver_con_limite_tiempo(modelo, tiempo_limite_segundos=2400, verbose=True):
    """
    Resuelve el modelo optimizado con límite de tiempo
    """
    if verbose:
        print(f"\n⏰ Resolviendo modelo optimizado: {tiempo_limite_segundos/60:.1f} minutos")
        print(f"📍 Almacén: Polígono 18 → 5 destinos más cercanos")
    
    solver = pyo.SolverFactory("glpk")
    tiempo_inicio = time.time()
    
    resultado = solver.solve(
        modelo, 
        tee=verbose,
        options=[f'--tmlim', str(tiempo_limite_segundos)]
    )
    
    tiempo_transcurrido = time.time() - tiempo_inicio
    
    print(f"\n📊 RESULTADO DESPUÉS DE {tiempo_transcurrido/60:.1f} MINUTOS:")
    print("="*60)
    
    if resultado.solver.status == SolverStatus.ok:
        if resultado.solver.termination_condition == TerminationCondition.optimal:
            print(f"✅ SOLUCIÓN ÓPTIMA ENCONTRADA")
        elif resultado.solver.termination_condition == TerminationCondition.maxTimeLimit:
            print(f"⏰ TIEMPO LÍMITE ALCANZADO - MEJOR SOLUCIÓN ENCONTRADA")
        else:
            print(f"✅ SOLUCIÓN FACTIBLE ENCONTRADA")
            
        # Extraer resultados
        costo_total = pyo.value(modelo.OBJ)
        plantas_total = sum(pyo.value(modelo.E[s, p, t]) for s in modelo.S for p in modelo.P for t in modelo.T)
        viajes_total = sum(pyo.value(modelo.V[p, t]) for p in modelo.P for t in modelo.T)
        
        print(f"💰 Costo total: ${costo_total:,.2f}")
        print(f"🌱 Plantas plantadas: {int(plantas_total):,}")
        print(f"🚛 Viajes realizados: {int(viajes_total)} (desde polígono 18)")
        print(f"📊 Demanda cubierta: {plantas_total/demanda_total*100:.1f}%")
        
        # Análisis detallado
        print(f"\n📈 ANÁLISIS DETALLADO:")
        
        # Costos desglosados
        costo_compras = sum(pyo.value(modelo.C[s, j]) * pyo.value(modelo.X[s, j, t]) 
                           for s in modelo.S for j in modelo.J for t in modelo.T)
        costo_plantacion = 20 * plantas_total
        costo_viajes = 4500 * viajes_total
        
        print(f"   💵 Costo compras: ${costo_compras:,.2f} ({costo_compras/costo_total*100:.1f}%)")
        print(f"   🌱 Costo plantación: ${costo_plantacion:,.2f} ({costo_plantacion/costo_total*100:.1f}%)")
        print(f"   🚛 Costo viajes: ${costo_viajes:,.2f} ({costo_viajes/costo_total*100:.1f}%)")
        
        # Distribución por polígono (ordenado por cercanía)
        print(f"\n🗺️ DISTRIBUCIÓN POR POLÍGONO (ordenado por distancia):")
        poligonos_ordenados = sorted(poligonos_filtrados, key=lambda p: tiempo_almacen_poligono[p])
        
        for p in poligonos_ordenados:
            plantas_poligono = sum(pyo.value(modelo.E[s, p, t]) for s in modelo.S for t in modelo.T)
            viajes_poligono = sum(pyo.value(modelo.V[p, t]) for t in modelo.T)
            demanda_poligono = sum(demanda_filtrada.get((s, p), 0) for s in nombres_especies)
            tiempo_viaje = tiempo_almacen_poligono[p]
            
            if plantas_poligono > 0:
                print(f"   {p}: {int(plantas_poligono):,} plantas, {int(viajes_poligono)} viajes "
                      f"(dist: {tiempo_viaje:.4f}h, demanda: {demanda_poligono:,}, {plantas_poligono/demanda_poligono*100:.1f}%)")
        
        # Resumen de eficiencia
        print(f"\n⚡ ANÁLISIS DE EFICIENCIA:")
        tiempo_total_viajes = sum(tiempo_almacen_poligono[p] * 2 * 
                                 sum(pyo.value(modelo.V[p, t]) for t in modelo.T) 
                                 for p in poligonos_filtrados)
        print(f"   🕐 Tiempo total en viajes: {tiempo_total_viajes:.2f} horas")
        print(f"   📦 Promedio plantas/viaje: {plantas_total/max(viajes_total,1):.0f}")
        print(f"   💰 Costo promedio/planta: ${costo_total/plantas_total:.2f}")
        
        return {
            'status': 'success',
            'costo_total': costo_total,
            'plantas_total': int(plantas_total),
            'viajes_total': int(viajes_total),
            'tiempo_resolucion': tiempo_transcurrido,
            'es_optima': resultado.solver.termination_condition == TerminationCondition.optimal,
            'modelo': modelo
        }
        
    else:
        print(f"❌ NO SE ENCONTRÓ SOLUCIÓN FACTIBLE")
        print(f"Estado del solver: {resultado.solver.status}")
        
        return {
            'status': 'failed',
            'solver_status': resultado.solver.status,
            'termination_condition': resultado.solver.termination_condition,
            'tiempo_resolucion': tiempo_transcurrido
        }

# === ANÁLISIS COMPARATIVO DE DISTANCIAS ===
def analizar_optimizacion_distancias():
    """
    Analiza la optimización lograda con los polígonos más cercanos
    """
    print(f"\n🎯 ANÁLISIS DE OPTIMIZACIÓN LOGÍSTICA")
    print("="*60)
    
    # Datos originales vs optimizados
    poligonos_originales = ['poligono3', 'poligono4', 'poligono21', 'poligono9', 'poligono10']
    tiempos_originales = {'poligono3': 0.28, 'poligono4': 0.33, 'poligono9': 0.15, 
                         'poligono10': 0.20, 'poligono21': 0.75}
    
    print(f"📊 COMPARACIÓN POLÍGONOS ORIGINALES VS OPTIMIZADOS:")
    print(f"\n🔴 CONFIGURACIÓN ORIGINAL:")
    tiempo_promedio_original = np.mean(list(tiempos_originales.values()))
    for p, t in sorted(tiempos_originales.items(), key=lambda x: x[1]):
        print(f"   {p}: {t:.4f} horas")
    print(f"   📈 Tiempo promedio: {tiempo_promedio_original:.4f} horas")
    
    print(f"\n🟢 CONFIGURACIÓN OPTIMIZADA (5 MÁS CERCANOS):")
    tiempo_promedio_optimizado = np.mean(list(tiempo_almacen_poligono.values()))
    for p, t in sorted(tiempo_almacen_poligono.items(), key=lambda x: x[1]):
        print(f"   {p}: {t:.4f} horas")
    print(f"   📈 Tiempo promedio: {tiempo_promedio_optimizado:.4f} horas")
    
    # Cálculos de mejora
    mejora_porcentual = ((tiempo_promedio_original - tiempo_promedio_optimizado) / tiempo_promedio_original) * 100
    print(f"\n✅ MEJORAS LOGRADAS:")
    print(f"   ⚡ Reducción tiempo promedio: {mejora_porcentual:.1f}%")
    print(f"   🚀 Tiempo ahorrado por viaje: {(tiempo_promedio_original - tiempo_promedio_optimizado)*2:.4f}h ida+vuelta")
    
    # Proyección de ahorros
    viajes_estimados = np.ceil(demanda_total / 524)  # 524 plantas por viaje
    ahorro_total_horas = viajes_estimados * (tiempo_promedio_original - tiempo_promedio_optimizado) * 2
    ahorro_combustible = ahorro_total_horas * 50  # Estimación $50/hora combustible
    
    print(f"\n💰 PROYECCIÓN DE AHORROS (para {demanda_total:,} plantas):")
    print(f"   🚛 Viajes estimados: {viajes_estimados:.0f}")
    print(f"   ⏱️ Horas ahorradas: {ahorro_total_horas:.1f}")
    print(f"   💵 Ahorro combustible: ${ahorro_combustible:,.0f}")

# === RESOLVER Y ANALIZAR ===
if __name__ == "__main__":
    print(f"\n🚀 MODELO OPTIMIZADO - 5 POLÍGONOS MÁS CERCANOS")
    
    # Análisis de optimización
    analizar_optimizacion_distancias()
    
    # Crear y resolver modelo optimizado
    print(f"\n1️⃣ Resolviendo modelo optimizado...")
    modelo = crear_modelo_optimizacion()
    
    # Resolver con límite de 40 minutos
    resultado = resolver_con_limite_tiempo(modelo, tiempo_limite_segundos=2400, verbose=True)
    
    if resultado['status'] == 'success':
        print(f"\n🏆 MODELO OPTIMIZADO RESUELTO EXITOSAMENTE")
        print(f"📍 Almacén: POLÍGONO 18")
        print(f"🎯 Destinos optimizados: {', '.join([p.split('o')[1] for p in poligonos_filtrados])}")
        print(f"💰 Costo total: ${resultado['costo_total']:,.2f}")
        print(f"🌱 Plantas: {resultado['plantas_total']:,}")
        print(f"🚛 Viajes: {resultado['viajes_total']}")
        print(f"⏰ Tiempo resolución: {resultado['tiempo_resolucion']/60:.1f} minutos")
        
    else:
        print(f"❌ ERROR EN LA RESOLUCIÓN DEL MODELO OPTIMIZADO")

print(f"\n📋 RESUMEN DE OPTIMIZACIONES:")
print(f"   ✅ Polígonos seleccionados por cercanía al almacén (polígono 18)")
print(f"   ✅ Datos reales de demanda desde archivo Excel")
print(f"   ✅ Tiempos reales de viaje desde matriz de tiempos")
print(f"   ✅ Reducción significativa en tiempos de transporte")
print(f"   ✅ Modelo más eficiente y realista")